# Process Velmeshev et al.

## Set up Env

In [1]:
import os
import pandas as pd
import numpy as np
import scanpy as sc
from scipy.sparse import csr_matrix

In [2]:
import liana as li

In [3]:
from prep_utils import filter_samples, filter_celltypes, map_gene_symbols

In [4]:
dataset = 'velmeshev'
groupby = 'cluster'
sample_key = 'individual'
condition_key = 'diagnosis'
batch_key = 'sex'

min_cells_per_sample = 700
sample_zcounts_max = 3
sample_zcounts_min = -2

# set filtering parameters
min_cells = 10 # min number of cells per cell type
min_samples = 5 # min number of samples that pass the threshold per cell type

## Preprocess

### Load data

In [ ]:
adata = sc.read_h5ad(os.path.join('data', f"{dataset}.h5ad"))
adata

In [ ]:
adata.obs[['individual', 'diagnosis']].drop_duplicates().nunique()

### Convert to Genesymbols

In [ ]:
df = adata.var.reset_index()['index'].str.split('\\|', expand=True).rename(columns={0:'ensembl', 1:'genesymbol'})
adata.var = df.set_index('ensembl')
map_df = df.rename(columns={'ensembl':'alias', 'genesymbol':'gene'})
map_df

In [ ]:
adata = map_gene_symbols(adata, map_df)

In [ ]:
adata

Filter samples, cell types, and genes

In [ ]:
adata = filter_samples(adata, sample_key, condition_key, min_cells_per_sample, sample_zcounts_max, sample_zcounts_min)

In [ ]:
## ^ Double check why are there more than 16?

In [ ]:
adata = filter_celltypes(adata=adata, groupby=groupby, sample_key=sample_key, min_cells=min_cells, min_samples=min_samples)

In [ ]:
# Remove genes expressed in few cells, normalize
sc.pp.filter_genes(adata, min_cells=30)

### Normalize

In [ ]:
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)

## Run LIANA

In [ ]:
li.mt.rank_aggregate.by_sample(adata, groupby=groupby, use_raw=False, sample_key=sample_key, verbose=True, n_perms=None)

In [ ]:
dataset

### Add Metadata & Write

In [ ]:
adata.uns['sample_key'] = sample_key
adata.uns['batch_key'] = batch_key
adata.uns['condition_key'] = condition_key

In [ ]:
assert np.isin(['sample_key', 'batch_key', 'condition_key'], adata.uns_keys()).all()

In [ ]:
adata.write_h5ad(os.path.join('data', f"{dataset}_processed.h5ad"))

## Classify

In [5]:
import scanpy as sc
from classify_utils import classifier_pipe

/home/dbdimitrov/anaconda3/envs/spiana/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


In [6]:
adata = sc.read_h5ad(os.path.join('data', f"{dataset}_processed.h5ad"), backed='r')

In [7]:
classifier_pipe(adata, dataset)

Creating views with: lr_means


100%|██████████| 289/289 [00:54<00:00,  5.26it/s]



        #########################################################
        ###           __  __  ____  ______                    ### 
        ###          |  \/  |/ __ \|  ____/\    _             ### 
        ###          | \  / | |  | | |__ /  \ _| |_           ### 
        ###          | |\/| | |  | |  __/ /\ \_   _|          ###
        ###          | |  | | |__| | | / ____ \|_|            ###
        ###          |_|  |_|\____/|_|/_/    \_\              ###
        ###                                                   ### 
        ######################################################### 
       
 
        
Loaded view='Oligodendrocytes&Oligodendrocytes' group='group1' with N=40 samples and D=60 features...
Loaded view='Oligodendrocytes&Neu-mat' group='group1' with N=40 samples and D=91 features...
Loaded view='OPC&OPC' group='group1' with N=40 samples and D=183 features...
Loaded view='Neu-NRGN-II&Neu-NRGN-II' group='group1' with N=40 samples and D=75 features...
Loaded view='AST-

100%|██████████| 40/40 [05:48<00:00,  8.72s/it]


Running Tensor Factorization
Creating views with: expr_prod


100%|██████████| 289/289 [00:53<00:00,  5.43it/s]



        #########################################################
        ###           __  __  ____  ______                    ### 
        ###          |  \/  |/ __ \|  ____/\    _             ### 
        ###          | \  / | |  | | |__ /  \ _| |_           ### 
        ###          | |\/| | |  | |  __/ /\ \_   _|          ###
        ###          | |  | | |__| | | / ____ \|_|            ###
        ###          |_|  |_|\____/|_|/_/    \_\              ###
        ###                                                   ### 
        ######################################################### 
       
 
        
Loaded view='Oligodendrocytes&Oligodendrocytes' group='group1' with N=40 samples and D=60 features...
Loaded view='Oligodendrocytes&Neu-mat' group='group1' with N=40 samples and D=91 features...
Loaded view='OPC&OPC' group='group1' with N=40 samples and D=183 features...
Loaded view='Neu-NRGN-II&Neu-NRGN-II' group='group1' with N=40 samples and D=75 features...
Loaded view='AST-

100%|██████████| 40/40 [05:54<00:00,  8.87s/it]


Running Tensor Factorization
Creating views with: lr_logfc


In [ ]:
adata.uns['auc']